# Part 1: training the model

In [1]:
%load_ext autoreload
%autoreload 2
%reset_selective -f  .*(?<!_)$
from custom_imports import *
files = glob.glob('data/masked/*png')
masked_images_   = joblib.load('data/cleaned_muscles.pkl')
image_names = joblib.load("data/image_names.pkl")

/home/bzr0014/anaconda3/envs/tensorGpu2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/bzr0014/anaconda3/envs/tensorGpu2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/bzr0014/anaconda3/envs/tensorGpu2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/bzr0014/anaconda3/envs/te

In [2]:
def get_contour_features(cnt):
    area = cv2.contourArea(cnt)
    (sx, sy), (rw, rh), angle = cv2.minAreaRect(cnt)
    (cx, cy), cr = cv2.minEnclosingCircle(cnt)
    xm, ym = cnt.reshape(-1, 2).mean(axis=0)
    xmn, ymn = cnt.reshape(-1, 2).min(axis=0)
    xmx, ymx = cnt.reshape(-1, 2).max(axis=0)
    return [area/(rw*rh), xmn, ymn, xmx, ymx, cx, cy, cr, sx, sy, area, rw, rh, rw/rh, angle, xm, ym]
def get_contours_features(cnts):
    return np.stack([get_contour_features(cnt)+[len(cnts)] for cnt in cnts])
       
def get_contours_features_and_colors(img):
    if len(img.shape)>2:
        img = img.sum(axis=2)
    cnts = get_contours(img>0)
    labels = []
    for cnt in cnts:
        b, a = cnt.reshape(-1 ,2).T
        labels.append(img[(a, b)].mean())
    t = get_contours_features(cnts)
    t = (t-t.mean(axis=0, keepdims=True))/(t.std(axis=0, keepdims=True)+1e-9)
    return np.vstack([t.T, np.argsort(t, axis=0).T]).T, np.array(labels, dtype='uint8').reshape(-1, 1)

In [3]:
clssfd = dict(zip(files, map(cv2.imread, files)))
X = []
Y = []
for img0 in clssfd.values():
    x, y = get_contours_features_and_colors(img0)
    X.append(x)
    Y.append(y)
w = X[0].shape[1]
X = np.concatenate(X).reshape(-1, w)
Y_ = np.concatenate(Y).reshape(-1)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
#encoder = MultiLabelBinarizer()
#Y = encoder.fit_transform(Y_)
Y = Y_
Xtr, Xts, Ytr, Yts = train_test_split(X, Y, random_state=15, test_size=.6)

In [5]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=200, n_jobs=50, random_state=16)

In [6]:
model.fit(Xtr, Ytr)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=50, oob_score=False, random_state=16, verbose=0,
                       warm_start=False)

In [7]:
model.score(Xts, Yts), model.score(Xtr, Ytr)

(1.0, 1.0)

In [9]:
#encoder.inverse_transform(model.predict(x))
model.predict(x)

array([210, 200, 150, 230, 220, 240, 250, 180, 190], dtype=uint8)

In [11]:
muscle_colors = dict([x.split(',') for x in """Left banana  ,250
Right banana  ,240
Left central  ,230
Right central  ,220
Bottom left  ,210
Bottom right  ,200
Top left  ,190
Top right  ,180
Btw bottom and center left  ,170
Btw bottom and center right  ,160
Center  ,150
Left half banana  ,140
Right half banana  ,130""".split('\n')])
reverse_muscle_colors = {int(v): k for k, v in muscle_colors.items()}
print(muscle_colors)
print(reverse_muscle_colors)

{'Left banana  ': '250', 'Right banana  ': '240', 'Left central  ': '230', 'Right central  ': '220', 'Bottom left  ': '210', 'Bottom right  ': '200', 'Top left  ': '190', 'Top right  ': '180', 'Btw bottom and center left  ': '170', 'Btw bottom and center right  ': '160', 'Center  ': '150', 'Left half banana  ': '140', 'Right half banana  ': '130'}
{250: 'Left banana  ', 240: 'Right banana  ', 230: 'Left central  ', 220: 'Right central  ', 210: 'Bottom left  ', 200: 'Bottom right  ', 190: 'Top left  ', 180: 'Top right  ', 170: 'Btw bottom and center left  ', 160: 'Btw bottom and center right  ', 150: 'Center  ', 140: 'Left half banana  ', 130: 'Right half banana  '}


In [12]:
n = np.random.randint(len(masked_images_))
multi_label_images = {}
for image_name, img in zip(image_names, masked_images_):
    img = img-1
    x, y = get_contours_features_and_colors(img)
    #pred = encoder.inverse_transform(model.predict(x))
    pred = model.predict(x)
    if len(np.unique(pred))  != len(pred):
        continue
    imt = np.zeros_like(img, dtype='uint8')
    for cnt, color in zip(get_contours(img), pred):
        color = int(color)
        #print(color, reverse_muscle_colors.get(color))
        cv2.drawContours(imt, [cnt], -1, color, -1)
    multi_label_images[image_name] = imt

In [15]:
#joblib.dump(multi_label_images, "data/multi_label_images.dict")

['data/multi_label_images.dict']